# More on Parallelization using Numba

In [1]:
import numpy as np
from numba import njit, prange
import quantecon as qe

### European Call Option

A European call option is a contract between the buyer and the seller of the call option to exchange a security at a set price. 

- The buyer of the option has the right, but not the obligation, to buy an agreed quantity of a particular commodity or financial instrument (the underlying) from the seller of the option *at the expiration date* for a certain price (the strike price). 

- The seller (or "writer") is obliged to sell the commodity or financial instrument to the buyer if the buyer so decides. 

The price of a call option obeys
$$
P = \beta^n \mathbb E \max \{ S_n - K, 0\}
$$
where
- $\beta$ : the discount factor
- $n$ : the expiration date
- $K$ : the strike price
- $S_t$ : the price of the underlying asset at time $t$

Assume that the price obeys
$$
\ln \frac{S_{t+1}}{S_t} = \mu + \sigma_t \xi_{t+1}, 
\quad \{\xi_t\} \overset{iid}{\sim} N(0,1) \\
\sigma_t = \exp(h_t), \quad
h_{t+1} = \rho h_t + \nu \eta_{t+1}, \quad
\{\eta_t\} \overset{iid}{\sim} N(0,1)
$$

where $S_0$ and $h_0$ are given. Define $s_t := \ln S_t$. Then 

$$
s_{t+1} = s_t + \mu + \sigma_t \xi_{t+1}
$$

By the law of large number, we can generate $M$ paths of $s_n$, denoted by $\{s_n^m\}_{m=1}^M$, and estimate the option price $P$ by 

$$
\hat P_M := \frac{1}{M} \sum_{m=1}^M \beta^n \max\{\exp(s_n^m) - K, 0\}
$$

#### References:
https://python-programming.quantecon.org/numba.html#exercises

In [2]:
@njit(parallel=True)
def call_option_price(β=0.99, 
                      μ=0.0001,
                      S0=10,
                      h0=0,
                      K=100,
                      n=20,
                      ρ=0.1,
                      v=0.001,
                      M=10_000_000,
                      seed=None):
    "Compute the price of the call option."
    #s_sim = np.empty(M)
    cumsum = 0
    for m in prange(M):
        if seed is not None:
            np.random.seed(seed+m)
        s = np.log(S0)
        h = h0
        for t in range(n):
            σ = np.exp(h)
            s = s + μ + σ*np.random.randn()
            h = ρ*h + v*np.random.randn()
        cumsum += np.maximum(np.exp(s)-K, 0)
        #s_sim[m] = s
    
    return (β**n) * cumsum / M

In [3]:
call_option_price(seed=1234)

133552.9572638895

In [6]:
with qe.Timer():
    call_option_price()

0.71 seconds elapsed
